# Java

Java is the mother of all JVM languages, first released in 1995 after years of development by Sun Microsystems.  BeakerX uses [OpenJDK](http://openjdk.java.net/) for Java and all the kernels.

BeakerX supports Java cells that define classes, which can be used by other JVM based cells.
One cell is equivalent to a Java compilation unit and can contain a single class, or a sequence of statements.

In [1]:
%classpath add mvn org.json json 20180813

In [2]:
import org.json.JSONObject;
import org.json.JSONArray;

import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;
import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Scanner;


        URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode("gare de Lyon", "UTF-8")+"&postcode=75000");
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            JSONArray res = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
    System.err.println(res);//longitude , latitude
        }


[2.3469,48.8589]


null

In [ ]:
package test.beaker;

import java.util.Date;
import java.text.SimpleDateFormat;

public class BeakerTest {
  private Date _date;
  private SimpleDateFormat sdf = new SimpleDateFormat("yyyy-MM-dd'T'HH:mmZ");
  
  public BeakerTest() {
    _date = new Date();
  }

  public String getDateTxt() {
    return "Today:" + sdf.format(_date);
  }

  public String getDateUpperCaseTxt() {
    return getDateTxt().toUpperCase();
  }

}


In [8]:
import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;

import org.json.JSONObject;
import org.json.JSONArray;

import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Map;
import java.util.HashMap;
import java.util.Scanner;

public class GeoPoint{
    private final double latitude;
    private final double longitude;
    private final Map<GeoPoint, Integer> neighborToDistance;
    public GeoPoint(double latitude, double longitude){
        this.latitude= latitude;
        this.longitude= longitude;
	neighborToDistance= new HashMap<GeoPoint, Integer>();
    }

    public double distanceTo(GeoPoint other){
        return distanceTo(other.latitude, other.longitude);
    }
   public double distanceTo(double latitude, double longitude){
       double earthRadius= 6371e3;// in meters
        double phi1=Math.toRadians(this.latitude);
        double phi2=Math.toRadians(latitude);
        double deltaPhi=Math.toRadians(latitude- this.latitude);
        double deltaLambda= Math.toRadians(longitude- this.longitude);
        double a= Math.sin(deltaPhi/2) * Math.sin(deltaPhi/2) +
            Math.cos(phi1) * Math.cos(phi2) * Math.sin(deltaLambda/2) * Math.sin(deltaLambda/2);
        double c= 2* Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
        return earthRadius * c;
    }
    public String toString(){
        return "GeoPoint: "+super.toString()+"{latitude: "+latitude+", longitude: " + longitude+", nb of neighbors"+neighborToDistance.size()+"}";
    }
    public static GeoPoint closerTo(double latitude, double longitude, double deltaD, Collection<GeoPoint> points){
        GeoPoint res=null;
        double minD= Double.POSITIVE_INFINITY;
        for(GeoPoint point : points){
            double currentD= point.distanceTo(latitude, longitude);
            if((currentD < minD) && (currentD > deltaD) ){
                minD= currentD;
                res= point;
            }
        }
        return res;
    }
    public static GeoPoint findByName(String name){
	    GeoPoint res= null;
        try{
            
            URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(name, "UTF-8")+"&postcode=75000");
            try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
                JSONArray coords = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
                res= new GeoPoint(coords.getDouble(0), coords.getDouble(1));
            }catch(Exception e){
                System.err.println(e);
            }
        }catch(Exception e){
            System.err.println(e);
        }  
    return res; // should throw
    }

    public static GeoPoint closerTo(String name, Collection<GeoPoint> data){
        return closerTo(findByName(name), data);
    }
    public static GeoPoint closerTo(GeoPoint ref, Collection<GeoPoint> data){
        return closerTo(ref.latitude, ref.longitude, 1.e-50, data);
    }
public static List<GeoPoint> read(String dataUrl) throws IOException {
        List<GeoPoint> res= new ArrayList<GeoPoint>();
        URL url= new URL(dataUrl);
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            String[] headers= br.readLine().split(" ");
            int nbVertices= Integer.parseInt(headers[0]);
            int nbEdges= Integer.parseInt(headers[1]);
            for(int i=0; i != nbVertices; ++i){
                String[] coords= br.readLine().split(" ");//latitude, longitude
                res.add(new GeoPoint(Double.parseDouble(coords[0]), Double.parseDouble(coords[1])));
            }
            for(int i=0; i != nbEdges; ++i){
                String[] data= br.readLine().split(" ");//start end nbDirs Duration Length
                GeoPoint start= res.get(Integer.parseInt(data[0]));
                GeoPoint end= res.get(Integer.parseInt(data[1]));
                Integer length= Integer.parseInt(data[4]);
                start.neighborToDistance.put(end, length);
                if(data[2].equals("2")){
                    end.neighborToDistance.put(start, length);
                }
            }
        }catch (Exception e){
            System.err.println(e);
        }
        return res;
    }
    public static void main(String[] args){
        try{
            String url="https://raw.githubusercontent.com/jilljenn/tryalgo/master/examples/paris.txt";
            List<GeoPoint> data= read(url);
            System.out.println(data);
        }catch(FileNotFoundException e){
            System.err.println(e);
        }
        catch(IOException e){
            System.err.println(e);
        }
    }
    
}

com.twosigma.beaker.javash.bkr98575a75.GeoPoint

In [9]:
String[] args={};
GeoPoint.main(args);

[GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@65f1bdd3{latitude: 48.8351503, longitude: 2.3077904, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@7948eab0{latitude: 48.8317022, longitude: 2.3579381, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@56b31af6{latitude: 48.8812439, longitude: 2.2917141, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@4ff69947{latitude: 48.8558991, longitude: 2.3550941, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@ef24ad0{latitude: 48.8406029, longitude: 2.3993259, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@4e680eef{latitude: 48.8587301, longitude: 2.4106012000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@79d0d6bb{latitude: 48.879117900000004, longitude: 2.3369039000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@72636c

2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@d9e349e{latitude: 48.8718031, longitude: 2.2935687000000002, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@6d2f920f{latitude: 48.859518, longitude: 2.3540808, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@3bbb9637{latitude: 48.8252916, longitude: 2.3626607, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@32fee060{latitude: 48.837623300000004, longitude: 2.3223905, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@1aa326e6{latitude: 48.831445800000004, longitude: 2.3807413, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@2a435840{latitude: 48.875665500000004, longitude: 2.3196156, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@530f0c35{latitude: 48.867839200000006, longitude: 2.3139027000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.b

sh.bkr98575a75.GeoPoint@12f440b4{latitude: 48.8687618, longitude: 2.3110097, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@29e40948{latitude: 48.875008, longitude: 2.3574993, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@5efc120e{latitude: 48.847566500000006, longitude: 2.3227914000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@e8d3e51{latitude: 48.818424900000004, longitude: 2.3699035, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@662e8f2f{latitude: 48.825801600000005, longitude: 2.371827, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@30cfeb6d{latitude: 48.829765300000005, longitude: 2.3570651000000002, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@1c580ba{latitude: 48.853041600000005, longitude: 2.2618851, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@132c02dc{lat

 of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@58c2896{latitude: 48.888485900000006, longitude: 2.3355041, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@647970a6{latitude: 48.8920808, longitude: 2.3462443, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@151cff82{latitude: 48.8589384, longitude: 2.4058751000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@14a22a6{latitude: 48.836274700000004, longitude: 2.2570944, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@181f6d10{latitude: 48.8845478, longitude: 2.3215339, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@135ab7d9{latitude: 48.8450283, longitude: 2.2976957000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@2e56739f{latitude: 48.848349600000006, longitude: 2.3760104, nb of neighbors3}, GeoPoint: com.twosigma.beaker.jav

@53d0f138{latitude: 48.837184400000005, longitude: 2.3001202000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@68545db3{latitude: 48.8574357, longitude: 2.3796886, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@77d5c716{latitude: 48.859144300000004, longitude: 2.3794671000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@18804f3d{latitude: 48.850145000000005, longitude: 2.3483419000000003, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@4d7371a1{latitude: 48.8871708, longitude: 2.3495686, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@562ecd93{latitude: 48.8294447, longitude: 2.3688264, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@607784cb{latitude: 48.87276790000001, longitude: 2.397056, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@1553b60c{latitude: 48.8726157000

...

03, nb of neighbors6}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@51817223{latitude: 48.8478567, longitude: 2.4151291, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@2922d9c9{latitude: 48.8775874, longitude: 2.4082945000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@312cfd16{latitude: 48.8481625, longitude: 2.3974807, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@525985e4{latitude: 48.889327200000004, longitude: 2.3370177, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@1ad03fbf{latitude: 48.8280303, longitude: 2.3924407000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@42d466ad{latitude: 48.8696671, longitude: 2.3264549000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@49c74307{latitude: 48.825754800000006, longitude: 2.3465866, nb of neighbors3}, GeoPoint: com.twosigma.b

ash.bkr98575a75.GeoPoint@6014dd47{latitude: 48.8535185, longitude: 2.3097623, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@e896904{latitude: 48.843245800000005, longitude: 2.383671, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@6922bde{latitude: 48.8421499, longitude: 2.2816381000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@2dadd361{latitude: 48.845419, longitude: 2.2558730000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@71cc64d6{latitude: 48.876739400000005, longitude: 2.2945361, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@6a34877c{latitude: 48.8869897, longitude: 2.3862934, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@261b0b26{latitude: 48.8580106, longitude: 2.2768551, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@6bd38ecf{latitude: 48.8328197, long

00000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@55e5b0d5{latitude: 48.859753700000006, longitude: 2.3519682, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@655ffd21{latitude: 48.861238500000006, longitude: 2.3606569, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@7b3ac992{latitude: 48.878907700000006, longitude: 2.3782625, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@3e0f3998{latitude: 48.8859937, longitude: 2.3892805000000004, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@5f0ad6d5{latitude: 48.833441, longitude: 2.3657363, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@55b393b3{latitude: 48.865756100000006, longitude: 2.3568701, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@216bd18f{latitude: 48.8816007, longitude: 2.2923681, nb of neighbors2}, GeoPoint: com.twosigm

eaker.javash.bkr98575a75.GeoPoint@72bc98ac{latitude: 48.840714500000004, longitude: 2.2682241000000003, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@1f67ec6e{latitude: 48.8675926, longitude: 2.3415172, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@d1a1925{latitude: 48.852675500000004, longitude: 2.3387938000000004, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@3ce05551{latitude: 48.8585421, longitude: 2.3678391000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@9b4882d{latitude: 48.8845286, longitude: 2.359327, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@495a3888{latitude: 48.884465500000005, longitude: 2.3494898, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@2c472ada{latitude: 48.82958360000001, longitude: 2.3689668, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr98575a75.GeoPoint@7

null

In [ ]:
package test.beaker;

BeakerTest bt = new BeakerTest();
return bt.getDateTxt();

In [ ]:
import java.util.List;
import java.util.ArrayList;
import com.twosigma.beakerx.chart.xychart.Plot;
import com.twosigma.beakerx.chart.xychart.plotitem.*;
import com.twosigma.beakerx.chart.Color;

Plot p = new Plot();

p.setTitle("this is a Java plot");

Bars b = new Bars();

List<Number> yList = new ArrayList<Number>();
yList.add(2);
yList.add(5);
yList.add(4);
yList.add(8);

b.setY(yList);
b.setColor(Color.blue);
b.setWidth(0.5);

p.add(b);
  
return p;

In [ ]:
package test.beaker;
interface DateGetter {
   public String getDateTxt();
}

In [ ]:
package test.beaker;
public class DG2 extends BeakerTest implements DateGetter {
}

## Using the classpath to load a jar

In [ ]:
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

In [ ]:
System.out.println("The Groovy working folder is :");
System.out.println(java.nio.file.Paths.get(".").toAbsolutePath().normalize().toString());
System.out.println("BeakerXClasspathTest.jar exists in this folder");

In [ ]:
import com.beaker.BeakerXClasspathTest;

BeakerXClasspathTest t = new BeakerXClasspathTest();
System.out.println(com.beaker.BeakerXClasspathTest.staticTest);
System.out.println(t.getObjectTest());